<a href="https://colab.research.google.com/github/ta269uec/semantic-search/blob/develop/k_means_sentence_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing all the packages necessary + loading the data

In [15]:
#!pip install datasets transformers torch sentence-transformers --quiet

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
LIB_LOCATION = "/content/drive/MyDrive/git-repos/semantic-search/"

In [4]:
!git clone -b develop https://github.com/ta269uec/semantic-search.git /content/drive/MyDrive/git-repos/semantic-search/

Cloning into '/content/drive/MyDrive/git-repos/semantic-search'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 4), reused 7 (delta 1), pack-reused 0
Unpacking objects: 100% (19/19), 14.85 KiB | 57.00 KiB/s, done.


In [16]:
import sys
sys.path.append(LIB_LOCATION)

In [19]:
from src.sentence_embedder import SentenceEmbedder
from data.sst2 import get_reviews_sst2

In [20]:
pos_sentences, neg_sentences = get_reviews_sst2()

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset sst2 downloaded and prepared to /root/.cache/huggingface/datasets/sst2/default/2.0.0/9896208a8d85db057ac50c72282bcb8fe755accc671a57dd8059d4e130961ed5. Subsequent calls will reuse this data.


Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

Filter:   0%|          | 0/67349 [00:00<?, ? examples/s]

In [21]:
st = SentenceEmbedder()
pos_embeddings, neg_embeddings = st.encode(pos_sentences), st.encode(neg_sentences)

RuntimeError: ignored

# Build K-Means Clusters

In [ ]:
from sklearn.cluster import KMeans
def cluster_k_means(corpus, corpus_embeddings, num_clusters):
  clustering_model = KMeans(n_clusters=num_clusters)
  clustering_model.fit(corpus_embeddings)
  cluster_assignment = clustering_model.labels_
  clustered_sentences = [[] for i in range(num_clusters)]
  clustered_embeddings = [[] for i in range(num_clusters)]
  for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    clustered_embeddings[cluster_id].append(corpus_embeddings[sentence_id])
  return clustered_sentences, clustered_embeddings

In [ ]:
N = 25
clustered_pos_sentences, clustered_pos_embeddings = cluster_k_means(corpus = positive_sentences,\
                                                            corpus_embeddings = positive_embeddings,\
                                                            num_clusters = N)
clustered_neg_sentences, clustered_neg_embeddings = cluster_k_means(corpus = negative_sentences,\
                                                            corpus_embeddings = negative_embeddings,\
                                                            num_clusters = N)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Sanity Check 1: Are clusters semantically similar?

In [ ]:
len(clustered_pos_embeddings)

5

In [ ]:
len(clustered_pos_embeddings[0][0])

384